In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sps

plt.rcParams['font.size'] = 11


# Dataset “Auto-MPG”

- Проверьте гипотезу, если средний MPG для USA такой же, как для Japan (t-тест).
- Проверьте гипотезу, если дисперсия MPG для USA такая же, как для Japan (F-тест или Левена).

In [2]:
columnNames = [
    'mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
    'acceleration', 'model_year', 'origin', 'car_name'
]

autoMpgDataFrame = pd.read_csv(
    'datasets/auto-mpg.data',
    delim_whitespace=True,
    names=columnNames,
    na_values='?'
)
autoMpgDataFrame = autoMpgDataFrame.dropna().reset_index(drop=True)

print('Размер выборки (всего):', len(autoMpgDataFrame))
display(autoMpgDataFrame.head())


Размер выборки (всего): 392


,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [3]:
mpgSeries = autoMpgDataFrame['mpg'].dropna()
usaSeries = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==1, 'mpg'].dropna()
japanSeries = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==3, 'mpg'].dropna()

def printSummary(series, label):
    print(f"\n--- {label} ---")
    print('N =', series.size, 'Mean =', series.mean(), 'SD =', series.std(ddof=1))

    shStat, shP = sps.shapiro(series)
    print('Shapiro-Wilk statistic =', round(shStat,6), ', p-value =', round(shP,6))

    muHat = series.mean()
    sigmaHat = series.std(ddof=1)
    ksStat, ksP = sps.kstest(series, 'norm', args=(muHat, sigmaHat))
    print('Kolmogorov-Smirnov statistic =', round(ksStat,6), ', p-value =', round(ksP,6))

printSummary(mpgSeries, 'All')
printSummary(usaSeries, 'USA')
printSummary(japanSeries, 'Japan')



--- All ---
N = 392 Mean = 23.445918367346938 SD = 7.8050074865717995
Shapiro-Wilk statistic = 0.96717 , p-value = 0.0
Kolmogorov-Smirnov statistic = 0.081779 , p-value = 0.009956

--- USA ---
N = 245 Mean = 20.0334693877551 SD = 6.440384098813329
Shapiro-Wilk statistic = 0.933697 , p-value = 0.0
Kolmogorov-Smirnov statistic = 0.11475 , p-value = 0.002883

--- Japan ---
N = 79 Mean = 30.450632911392404 SD = 6.090048069738324
Shapiro-Wilk statistic = 0.979735 , p-value = 0.241967
Kolmogorov-Smirnov statistic = 0.118217 , p-value = 0.202785


In [4]:
# Гипотеза о равенстве средних (t-тест)
tStatMeans, tPvalueMeans = sps.ttest_ind(usaSeries, japanSeries, equal_var=False)
print('\n=== Тест на равенство средних (Welch\'s t-test) ===')
print('t-statistic =', round(tStatMeans, 4), ', p-value =', round(tPvalueMeans, 6))

# Гипотеза о равенстве дисперсий (Levene тест)
leveneStat, levenePvalue = sps.levene(usaSeries, japanSeries)
print('\n=== Тест на равенство дисперсий (Levene) ===')
print('Levene statistic =', round(leveneStat, 4), ', p-value =', round(levenePvalue, 6))

# Альтернатива: F-тест для дисперсий
fStat = np.var(usaSeries, ddof=1) / np.var(japanSeries, ddof=1)
fPvalue = 2 * min(sps.f.cdf(fStat, len(usaSeries)-1, len(japanSeries)-1), 
                  sps.f.sf(fStat, len(usaSeries)-1, len(japanSeries)-1))
print('F-statistic =', round(fStat, 4), ', p-value =', round(fPvalue, 6))



=== Тест на равенство средних (Welch's t-test) ===
t-statistic = -13.0339 , p-value = 0.0

=== Тест на равенство дисперсий (Levene) ===
Levene statistic = 0.1616 , p-value = 0.687919
F-statistic = 1.1184 , p-value = 0.56902


# Dataset “Auto-MPG”

- Проверить гипотезы о том, что средний MPG одинаков для разных origin (попарно и все вместе).

In [5]:
print('Размер выборки:', len(autoMpgDataFrame))


Размер выборки: 392


In [6]:
# ANOVA для всех origin (равенство средних MPG)
originGroups = [group['mpg'].dropna().values for name, group in autoMpgDataFrame.groupby('origin')]
anovaF, anovaP = sps.f_oneway(*originGroups)
print('=== ANOVA для равенства средних MPG по всем origin ===')
print('F-statistic =', round(anovaF, 4), ', p-value =', round(anovaP, 6))

# Попарные t-тесты (Welch's)
print('\n=== Попарные t-тесты (Welch\'s) ===')
origins = sorted(autoMpgDataFrame['origin'].unique())
for i in range(len(origins)):
    for j in range(i+1, len(origins)):
        origin1 = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==origins[i], 'mpg'].dropna().values
        origin2 = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==origins[j], 'mpg'].dropna().values
        tStat, tP = sps.ttest_ind(origin1, origin2, equal_var=False)
        print(f'Origin {origins[i]} vs {origins[j]}: t = {round(tStat,4)}, p = {round(tP,6)}')


=== ANOVA для равенства средних MPG по всем origin ===
F-statistic = 96.6015 , p-value = 0.0

=== Попарные t-тесты (Welch's) ===
Origin 1 vs 2: t = -8.4311, p = 0.0
Origin 1 vs 3: t = -13.0339, p = 0.0
Origin 2 vs 3: t = -2.7075, p = 0.007637


# Dataset “Auto-MPG”

- Проверить гипотезы о равенстве распределений характеристик авто разных origin (KS-тест попарно).
- Проверьте гипотезы о равенстве средних (ANOVA) и дисперсий (Levene) различных характеристик авто разных origin.

In [7]:
features = ['displacement', 'horsepower', 'weight', 'acceleration']
origins = sorted(autoMpgDataFrame['origin'].unique())

print('Размер выборки:', len(autoMpgDataFrame))
print('Origin:', origins)
display(autoMpgDataFrame[features + ['origin']].head())


Размер выборки: 392
Origin: [1, 2, 3]


,displacement,horsepower,weight,acceleration,origin
0,307.0,130.0,3504.0,12.0,1
1,350.0,165.0,3693.0,11.5,1
2,318.0,150.0,3436.0,11.0,1
3,304.0,150.0,3433.0,12.0,1
4,302.0,140.0,3449.0,10.5,1


In [8]:
for feature in features:
    print(f'\n=== {feature.upper()} ===')
    
    # ANOVA для средних
    groups = [autoMpgDataFrame.loc[autoMpgDataFrame['origin']==org, feature].dropna().values for org in origins]
    anovaF, anovaP = sps.f_oneway(*groups)
    print('ANOVA (средние): F =', round(anovaF, 4), ', p =', round(anovaP, 6))
    
    # Levene для дисперсий
    leveneStat, leveneP = sps.levene(*groups)
    print('Levene (дисперсии): stat =', round(leveneStat, 4), ', p =', round(leveneP, 6))



=== DISPLACEMENT ===
ANOVA (средние): F = 147.1241 , p = 0.0
Levene (дисперсии): stat = 106.071 , p = 0.0

=== HORSEPOWER ===
ANOVA (средние): F = 61.3421 , p = 0.0
Levene (дисперсии): stat = 26.5266 , p = 0.0

=== WEIGHT ===
ANOVA (средние): F = 112.772 , p = 0.0
Levene (дисперсии): stat = 42.759 , p = 0.0

=== ACCELERATION ===
ANOVA (средние): F = 14.9618 , p = 1e-06
Levene (дисперсии): stat = 4.2724 , p = 0.014609


In [9]:
# Попарные KS-тесты для распределений
for feature in features:
    print(f'\n=== KS-тесты для {feature.upper()} (попарно) ===')
    for i in range(len(origins)):
        for j in range(i+1, len(origins)):
            group1 = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==origins[i], feature].dropna().values
            group2 = autoMpgDataFrame.loc[autoMpgDataFrame['origin']==origins[j], feature].dropna().values
            ksStat, ksP = sps.ks_2samp(group1, group2)
            print(f'{origins[i]} vs {origins[j]}: D = {round(ksStat,4)}, p = {round(ksP,6)}')



=== KS-тесты для DISPLACEMENT (попарно) ===
1 vs 2: D = 0.7354, p = 0.0
1 vs 3: D = 0.7645, p = 0.0
2 vs 3: D = 0.205, p = 0.077219

=== KS-тесты для HORSEPOWER (попарно) ===
1 vs 2: D = 0.4813, p = 0.0
1 vs 3: D = 0.5028, p = 0.0
2 vs 3: D = 0.1718, p = 0.199088

=== KS-тесты для WEIGHT (попарно) ===
1 vs 2: D = 0.5019, p = 0.0
1 vs 3: D = 0.6767, p = 0.0
2 vs 3: D = 0.2206, p = 0.046618

=== KS-тесты для ACCELERATION (попарно) ===
1 vs 2: D = 0.2775, p = 0.000401
1 vs 3: D = 0.26, p = 0.000466
2 vs 3: D = 0.2373, p = 0.026082


# Dataset “Blood Pressure”

- Проверить гипотезу об успешности "лечения" с вероятностью 0.7 (0.8). Под успехом мы подразумеваем, что давление после вмешательства (bp_after) меньше, чем до (bp_before).

In [10]:
bpDataFrame = pd.read_csv('datasets/blood_pressure.csv')
display(bpDataFrame.head())

bpClean = bpDataFrame.dropna(subset=['bp_before', 'bp_after']).copy()

bpClean['success'] = (bpClean['bp_after'] < bpClean['bp_before'])  # Успех: давление после < до

numPatients = bpClean.shape[0]
numSuccesses = int(bpClean['success'].sum())

print('Количество пациентов с полными данными =', numPatients)
print('Количество успехов =', numSuccesses)

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


Количество пациентов с полными данными = 120
Количество успехов = 73


In [ ]:
# Биномиальный тест для p=0.5
binomTest05 = sps.binomtest(numSuccesses, numPatients, p=0.5, alternative='two-sided')
print('=== Биномиальный тест для p=0.5 ===')
print('statistic =', binomTest05.statistic, ', p-value =', round(binomTest05.pvalue, 6))

# Биномиальный тест для p=0.6
binomTest06 = sps.binomtest(numSuccesses, numPatients, p=0.6, alternative='two-sided')
print('\n=== Биномиальный тест для p=0.6 ===')
print('statistic =', binomTest06.statistic, ', p-value =', round(binomTest06.pvalue, 6))

# Биномиальный тест для p=0.7
binomTest07 = sps.binomtest(numSuccesses, numPatients, p=0.7, alternative='two-sided')
print('\n=== Биномиальный тест для p=0.7 ===')
print('statistic =', binomTest07.statistic, ', p-value =', round(binomTest07.pvalue, 6))

# Биномиальный тест для p=0.8
binomTest08 = sps.binomtest(numSuccesses, numPatients, p=0.8, alternative='two-sided')
print('\n=== Биномиальный тест для p=0.8 ===')
print('statistic =', binomTest08.statistic, ', p-value =', round(binomTest08.pvalue, 6))

=== Биномиальный тест для p=0.5 ===
statistic = 0.6083333333333333 , p-value = 0.022083

=== Биномиальный тест для p=0.6 ===
statistic = 0.6083333333333333 , p-value = 0.925825

=== Биномиальный тест для p=0.7 ===
statistic = 0.6083333333333333 , p-value = 0.035885

=== Биномиальный тест для p=0.8 ===
statistic = 0.6083333333333333 , p-value = 2e-06
